In [ ]:
from timeit import timeit

import numpy as np

from scipy import stats
from ebnmpy.estimators import estimators

In [ ]:
def sample_point_t(n, pi0=.8, df=5, scale=1.5):
    not_delta = stats.bernoulli.rvs(pi0, size=n) == 0
    z = np.zeros(n)
    z[not_delta] = stats.t.rvs(df=df, scale=scale, size=not_delta.sum())
    return z

In [ ]:
np.random.seed(0)
s = 1
n_repetitions = 5

n_list = (10 ** np.arange(2, 5.5, .5)).astype(int)
results = []

for n in n_list:
    x = sample_point_t(n) + stats.norm.rvs(size=n)

    for cls in estimators.values():
        t = timeit(lambda: cls().fit(x=x, s=s), number=n_repetitions) / n_repetitions
        results.append((cls.__name__, n, t))

In [ ]:
import pandas as pd

df = pd.DataFrame(results, columns=("ebnmpy class", "Number of observations", "Time elapsed (s)"))
df["Time elapsed (s)"] = df["Time elapsed (s)"]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use("ggplot")
plt.rcParams["axes.facecolor"] = "w"
plt.rcParams["grid.color"] = "lightgray"
plt.rcParams["axes.labelcolor"] = "k"
plt.rcParams["font.sans-serif"] = "Arial"
plt.rcParams["xtick.labelcolor"] = "k"
plt.rcParams["xtick.major.size"] = 0
plt.rcParams["ytick.major.size"] = 0
plt.rcParams["xtick.minor.size"] = 0
plt.rcParams["ytick.minor.size"] = 0

In [ ]:
fig = plt.figure(figsize=(4, 3))
ax = sns.lineplot(data=df, x="Number of observations", y="Time elapsed (s)", hue="ebnmpy class", marker=".", markersize=10,
                  palette=sns.palettes.husl_palette(len(estimators)), mew=0)
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend(bbox_to_anchor=(1, .5), loc="center left", frameon=False, numpoints=1)
ax.set_xticks(n_list, minor=True)
ax.set_yticks(10 ** np.array([-.5, -1.5, -2.5]), minor=True)
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
for h in ax.legend_.legendHandles: 
    h.set_marker('.')
    h.set_markersize(10)
fig.savefig("figure2.png", dpi=300, bbox_inches="tight", pad_inches=0)